In [1]:
# from keras_segmentation.models import resnet50 as rs
# from keras_segmentation.models.unet import resnet50_unet


In [2]:
import tensorflow as tf
from tensorflow import keras
from skimage.io import imread, imshow
from skimage.transform import resize
from tqdm import tqdm 
#import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
#from crfrnn_layer import CrfRnnLayer
from keras.models import Model
from image_segmentation_keras.keras_segmentation.models.model_utils import get_segmentation_model
from keras.layers import Conv2D, MaxPooling2D, Input, ZeroPadding2D, \
    Dropout, Conv2DTranspose, Cropping2D, Add, UpSampling2D
from keras.layers.merge import concatenate

from keras.preprocessing.image import ImageDataGenerator
from glob import glob
from keras.models import *
from keras.layers import *
from keras import backend as keras
from keras import Model
import sys
sys.path.insert(1, './src')
from crfrnn_model import get_crfrnn_model_def
from crfrnn_layer import CrfRnnLayer
from tensorflow import keras
from tensorflow.keras import layers

Using TensorFlow backend.


In [8]:
channels, height, width = 3, 512, 512


In [9]:
def _unet(n_classes, encoder, l1_skip_conn=True, input_height=416,
          input_width=608):

    img_input, levels = encoder(
        input_height=input_height, input_width=input_width)
    [f1, f2, f3, f4, f5] = levels
    print('f5',f5.shape)
    o = f5
#     print('f4',f4.shape)

    o = (BatchNormalization())(o)
    o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
    o = (Conv2D(512, (3, 3), padding='same', activation='relu' , data_format=IMAGE_ORDERING))(o)
    o = (Conv2D(512, (3, 3), padding='same', activation='relu' , data_format=IMAGE_ORDERING))(o)
    o = (BatchNormalization())(o)
    o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
    print('first upsample',o.shape)
    o = (concatenate([o, f3], axis=MERGE_AXIS))
    print('first concat',o.shape)
    o = (Conv2D(256, (3, 3), padding='same', activation='relu' , data_format=IMAGE_ORDERING))(o)
    o = (Conv2D(256, (3, 3), padding='same', activation='relu' , data_format=IMAGE_ORDERING))(o)
    o = (BatchNormalization())(o)
    print(o.shape)

    o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
    print('second up',o.shape)
    print('f3',f3.shape)
    o = (concatenate([o, f2], axis=MERGE_AXIS))
    print('2nd concat',o.shape)
    o = (Conv2D(128, (3, 3), padding='same' , activation='relu' , data_format=IMAGE_ORDERING))(o)
    o = (Conv2D(128, (3, 3), padding='same' , activation='relu' , data_format=IMAGE_ORDERING))(o)
    o = (BatchNormalization())(o)

    o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
    print('third up',o.shape)
    print('f2',f2.shape)
    o = (concatenate([o, f1], axis=MERGE_AXIS))
    print('third concat',o.shape)

    o = (Conv2D(64, (3, 3), padding='same', activation='relu', data_format=IMAGE_ORDERING))(o)
    o = (Conv2D(64, (3, 3), padding='same', activation='relu', data_format=IMAGE_ORDERING))(o)
    o = (BatchNormalization())(o)
    o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)

    o = Conv2D(n_classes, (3, 3), padding='same',
               data_format=IMAGE_ORDERING)(o)
    print("last upsample",o.shape)
    crf_output = CrfRnnLayer(image_dims=(height, width),
                         num_classes=n_classes,
                         theta_alpha=160.,
                         theta_beta=3.,
                         theta_gamma=3.,
                         num_iterations=10,
                         name='crfrnn')([o, img_input])
    model = get_segmentation_model(img_input, crf_output)

    return model

In [10]:
from keras_segmentation.models.config import IMAGE_ORDERING




if IMAGE_ORDERING == 'channels_first':
    MERGE_AXIS = 1
elif IMAGE_ORDERING == 'channels_last':
    MERGE_AXIS = -1
def get_vgg_encoder(input_height=224,  input_width=224, pretrained='imagenet'):

    assert input_height % 32 == 0
    assert input_width % 32 == 0

    if IMAGE_ORDERING == 'channels_first':
        img_input = Input(shape=(3, input_height, input_width))
    elif IMAGE_ORDERING == 'channels_last':
        img_input = Input(shape=(input_height, input_width, 3))

    x = Conv2D(64, (3, 3), activation='relu', padding='same',
               name='block1_conv1', data_format=IMAGE_ORDERING)(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same',
               name='block1_conv2', data_format=IMAGE_ORDERING)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool',
                     data_format=IMAGE_ORDERING)(x)
    f1 = x
    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same',
               name='block2_conv1', data_format=IMAGE_ORDERING)(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same',
               name='block2_conv2', data_format=IMAGE_ORDERING)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool',
                     data_format=IMAGE_ORDERING)(x)
    f2 = x

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same',
               name='block3_conv1', data_format=IMAGE_ORDERING)(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same',
               name='block3_conv2', data_format=IMAGE_ORDERING)(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same',
               name='block3_conv3', data_format=IMAGE_ORDERING)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool',
                     data_format=IMAGE_ORDERING)(x)
    f3 = x

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same',
               name='block4_conv1', data_format=IMAGE_ORDERING)(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same',
               name='block4_conv2', data_format=IMAGE_ORDERING)(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same',
               name='block4_conv3', data_format=IMAGE_ORDERING)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool',
                     data_format=IMAGE_ORDERING)(x)
    f4 = x

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same',
               name='block5_conv1', data_format=IMAGE_ORDERING)(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same',
               name='block5_conv2', data_format=IMAGE_ORDERING)(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same',
               name='block5_conv3', data_format=IMAGE_ORDERING)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool',
                     data_format=IMAGE_ORDERING)(x)
    f5 = x

#     if pretrained == 'imagenet':
#         VGG_Weights_path = keras.utils.get_file(
#             pretrained_url.split("/")[-1], pretrained_url)
#         Model(img_input, x).load_weights(VGG_Weights_path)

    return img_input, [f1, f2, f3, f4, f5]

In [6]:
def vgg_unet(n_classes, input_height=416, input_width=608, encoder_level=3):

    model = _unet(n_classes, get_vgg_encoder,
                  input_height=input_height, input_width=input_width)
    model.model_name = "vgg_unet"
    return model

In [7]:
model = vgg_unet(n_classes=2,input_height=512, input_width=512)



f5 (None, 16, 16, 512)
first upsample (None, 64, 64, 512)
first concat (None, 64, 64, 768)
(None, 64, 64, 256)
second up (None, 128, 128, 256)
f3 (None, 64, 64, 256)
2nd concat (None, 128, 128, 384)
third up (None, 256, 256, 128)
f2 (None, 128, 128, 128)
third concat (None, 256, 256, 192)
last upsample (None, 512, 512, 2)


ValueError: Dimensions must be equal, but are 512 and 256 for '{{node crfrnn/truediv}} = RealDiv[T=DT_FLOAT](crfrnn/HighDimFilter_2, crfrnn/HighDimFilter)' with input shapes: [2,512,512], [2,256,256].

In [ ]:
model.train(train_images = "/Users/mavaylon/Research/Research_Gambier/Data/BP_C_train/img/",
            train_annotations="/Users/mavaylon/Research/Research_Gambier/Data/BP_C_train/ann/",
            epochs=20,
            validate=True,
            steps_per_epoch=len(glob.glob("/Users/mavaylon/Research/Research_Gambier/Data/BP_C_train/img/*")),
            val_images="/Users/mavaylon/Research/Research_Gambier/Data/BP_C_test/img/",
            val_annotations="/Users/mavaylon/Research/Research_Gambier/Data/BP_C_test/ann/",
            val_steps_per_epoch=len(glob.glob("/Users/mavaylon/Research/Research_Gambier/Data/BP_C_test/img/*")),
            val_batch_size=1,
            #auto_resume_checkpoint=True,
            batch_size=1)

In [ ]:
import glob

img_names = sorted(glob.glob("/Users/mavaylon/Downloads/Equalized/*.png"))

for name in img_names:
    out_name = "/Users/mavaylon/Research/Research_Gambier/bp_castle_sandstone_vgg_unet_crf/" + name.split('/')[-1]
    print(out_name)
    out = model.predict_segmentation(inp=name, out_fname=out_name)